In [1]:
import wandb
# import util
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import shutil
import wandb
from PIL import Image
import os
import glob
from pathlib import Path

from tqdm.notebook import tqdm
from skimage.io import imread,imsave 

In [2]:
%reload_ext autoreload
%autoreload 2

from bioblue.nb.load import load_from_dir, load_from_dir2
# from bioblue.nb import *

In [5]:

#T400-T350_alternating
run_dir = Path("/globalscratch/users/n/s/nsayez/outputs/2022-06-22/11-27-27")

output_dir = '/globalscratch/users/n/s/nsayez/Classification_dataset/2002-2019/T400-T350-Alternating'

## Run the prediction on the images of the Classification Dataset

In [25]:
dataset_overrides = [
    'dataset.data_dir="/globalscratch/users/n/s/nsayez/Classification_dataset"',
    'dataset.dataset_name="2002-2019"',
    'dataset.train_dataset.partition="1024"',
    'dataset.val_dataset.partition="1024"',
    'dataset.test_dataset.partition="1024"',
    'dataset.train_dataset.dtypes=["image"]',
    'dataset.val_dataset.dtypes=["image"]',
    'dataset.test_dataset.dtypes=["image"]',
    
]
logger_overrides = [
    
]


In [12]:
config, model, dm, trainer = load_from_dir2(
                                    run_path= run_dir,
                                    load_trainer=True,
                                    override = 
                                                dataset_overrides + logger_overrides + [
#                                         '~callbacks.wandb',
                                        f'callbacks.Save_prediction.output_dir={output_dir}'
                                    ]
                                )
test_dataloader = dm.test_dataloader()

results = trainer.predict(model, datamodule=dm)
# # results = trainer.test(model, dataloaders=test_dataloader)

bioblue.loss.CombineLosses
['sunspot']
{'_target_': 'bioblue.dataset.BioblueDataModule', 'data_dir': '/globalscratch/users/n/s/nsayez/Classification_dataset', 'directory': 'synthetic', 'train_size': 50, 'val_size': 50, 'test_size': 50, 'shape': [512, 512], 'batch_size': 16, 'points_range': [20, 21], 'links_range': [3, 4], 'max_shapes': 1000, 'size_range': [50, 500], 'weight_range': [1, 5], 'bg_intensity_range': [0, 50], 'fg_intensity_range': [0, 256], 'num_workers': 12, 'dataset_name': '2002-2019', 'train_dataset': {'_target_': 'bioblue.dataset.DeepsunSegmentationDataset', 'partition': '1024', 'dtypes': ['image'], 'transforms': [{'_target_': 'bioblue.transforms.DeepsunRandomFlip'}, {'_target_': 'bioblue.transforms.DeepsunRandomRotate'}, {'_target_': 'bioblue.transforms.DeepsunCropNonEmptyMaskIfExists', 'height': 512, 'width': 512}, {'_target_': 'bioblue.transforms.DeepsunRandomMaskSelector'}]}, 'val_dataset': {'_target_': 'bioblue.dataset.DeepsunSegmentationDataset', 'partition': '1024

There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
The `on_init_end` callback hook was deprecated in v1.6 and will be removed in v1.8.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

In [5]:
wandb.finish()

epoch,▁
test_bgiou,▁
test_loss,▁
test_meaniou,▁
test_sunspotiou,▁
trainer/global_step,▁
epoch,0
test_bgiou,0.99976
test_loss,0.0006
test_meaniou,0.87409
test_sunspotiou,0.74843


## View Predictions

In [7]:
seg_types = ['GroundTruth_single']

# dataset_root = "/globalscratch/users/n/s/nsayez/deepsun_bioblue/All/test_GT"
dataset_root = "/globalscratch/users/n/s/nsayez/Classification_dataset/2002-2019/"

image_folder = os.path.join(dataset_root, 'all')
image_lst = sorted(glob.glob(os.path.join(image_folder, '*.FTS')))

run_folders = { f"{output_dir.split('/')[-1]}": dataset_root}

pred_folders = {t: os.path.join(run_folders[t] , t) for t in run_folders.keys()}
pred_lsts = {t : sorted(glob.glob(os.path.join(pred_folders[t], '*.png'))) for t in run_folders.keys()}


print(len(image_lst))
print(len(pred_lsts[list(pred_lsts.keys())[0]]))

3750
3750


In [8]:
%matplotlib ipympl

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import ipywidgets as widgets

cmap_pred = cm.cool
cmap_pred = cmap_pred(range(255))
cmap_pred = ListedColormap([(0, 0, 0, 0), *cmap_pred])


def refresh(event):
#     print(type(event['owner']))
    if type(event["owner"]) == widgets.widget_bool.Checkbox:
        xlims0 = axes0[0].get_xlim()
        ylims0 = axes0[0].get_ylim()
    
    axes0[0].clear()
    
    test_img = imread(image_lst[idx_slider.value])
    basename = os.path.basename(image_lst[idx_slider.value])
    axes0[0].set_title(basename)
    
    if img_cb.value:
        axes0[0].imshow(test_img, cmap="gray", interpolation="None")
    
    for i, cb in enumerate(pred_cbs):
        if cb.value: 
            tmp = imread(pred_lsts[list(pred_folders.keys())[i]][idx_slider.value])
            axes0[0].imshow(tmp, cmap=cmap_pred, interpolation="None", alpha=.5)  
#             axes0[0].imshow(tmp, interpolation="None")  

       
    if type(event["owner"]) == widgets.widget_bool.Checkbox:  
        if xlims0 != (0.0, 1.0):
            axes0[0].set_xlim(xlims0)
            axes0[0].set_ylim(ylims0)
    return

max_rows = 1
max_cols = 1

plt.ioff()
fig_widget0, axes0 = plt.subplots(nrows=max_rows, ncols=max_cols, figsize=(7,7))

try:
    len(axes0)
except TypeError:
    axes0 = [axes0]

plt.ion()
img_cb = widgets.Checkbox(value=True, description='Show img')
pred_cbs = [widgets.Checkbox(value=False, description=f'PRED {t}') for t in pred_folders.keys()]

idx_slider = widgets.IntSlider(value=0, min=0, max=len(image_lst)-1, step=1, description="Image Index")


# Input image to predict
test_img = imread(image_lst[0])

axes0[0].imshow(test_img, cmap="gray", interpolation="None")

img_cb.observe(refresh, names='value')
for cb in pred_cbs:
    cb.observe(refresh, names='value')
idx_slider.observe(refresh, names='value')

widgets.VBox([idx_slider,widgets.HBox([ img_cb]+pred_cbs), fig_widget0.canvas])

In [ ]:
# images intéressantes:
#     142

# images_pourries: 
69,75,93
94,201,203,311,315,337,341,403,409,420,441,613,615,668,726,743,755,778,779,976,996